In [1]:
#Restart kernel if rerunning this cell
import os 
wd = os.path.dirname(os.getcwd()).replace('\\', '/')
os.chdir(wd)

import pandas as pd
import geopandas as gpd
import scripts.preprocess_helper as ph
import scripts.cleanup_helper as ch
import omegaconf
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [2]:
yaml_path = os.path.join(wd, 'conf', 'config.yaml')
cfg = omegaconf.OmegaConf.load(yaml_path)
paths = cfg.paths
consts = cfg.constants

In [ ]:
%%time
# Load the raw data
utils_raw = ph.load_data(paths.utilities.util_shape_file_path, 'shp')
j40_raw = ph.load_data(paths.j40.j40_shp_path, 'shp')
coal_raw = ph.load_data(paths.energy.coal_shp_path, 'shp')
ffe_raw = ph.load_data(paths.energy.ffe_shp_path, 'shp')
states_fips = ph.load_data(paths.boundaries.state_fips_path, 'csv')
ct_raw = ph.load_data(paths.boundaries.ct_path, 'shp')
st_raw = ph.load_data(paths.boundaries.st_path, 'shp')

In [ ]:
%%time
#Data Cleaning using the helper files
util_clean, rural_coops, municipal_utils = ch.coops_utils_cleanup(util_shp = utils_raw, state_csv_df=states_fips, consts = consts.utilities)
j40_clean = ch.j40_cleanup(j40_shp = j40_raw.copy(), consts = consts.j40)
coal_clean, ffe_clean = ch.energy_cleanup(coal_shp = coal_raw.copy(), ffe_shp = ffe_raw.copy(), consts = consts.energy)
lic_clean = ch.low_inc_cleanup(pov_csv_path = paths.low_income.pov_dir, li_tract_csv_path= paths.low_income.tract_inc_dir, \
    li_st_csv_path=paths.low_income.st_inc_dir, li_msa_csv_path= paths.low_income.msa_inc_dir, \
        tracts_shp_path= paths.low_income.tract_shp_path ,msa_shp_path = paths.low_income.msa_shape_path , consts= consts.low_income)
county_df, state_df = ch.cty_st_borders_cleanup(county_df = ct_raw.copy(), st_fips_csv= states_fips, st_df = st_raw.copy(),\
     consts = consts.ct_st_borders)